In [1]:
import pandas as pd
import numpy as np
import keras
import gensim
import pickle
import gc
%matplotlib inline

%load_ext autoreload
%autoreload 2
import helper

Using TensorFlow backend.


In [2]:
X_train = np.load("data/word_vectors/pubmed_wiki_X_train_seeds42.npy")
y_train = np.load("data/word_vectors/pubmed_wiki_y_train_seeds42.npy")

In [3]:
X_test = np.load("data/word_vectors/pubmed_wiki_X_test_seeds42.npy")
y_test = np.load("data/word_vectors/pubmed_wiki_y_test_seeds42.npy")

In [4]:
X_train = np.concatenate((X_train, X_test), axis=0)
y_train = np.concatenate((y_train, y_test), axis=0)

gc.collect()

12

In [6]:
X_train.shape

(45587, 100, 200)

In [12]:
input_shape = (100, 200)

# Basic FeedForward Net (concat)

In [85]:
X_train_concat = X_train.reshape(X_train.shape[0], 100*200)
X_test_concat = X_test.reshape(X_test.shape[0], 100*200)

In [86]:
print(X_train_concat.shape)
print(X_test_concat.shape)

(41028, 20000)
(4559, 20000)


In [87]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(128, input_shape=(X_train_concat.shape[1],)))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(64))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(23, activation='softmax'))

In [88]:
model.compile(optimizer="adagrad",
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
print(model.summary())
model.fit(X_train_concat, y_train, validation_split=0.1, epochs=15, batch_size=256)
yhat = model.predict(X_test_concat)
_ = helper.score_prediction(y_test, yhat, binary=False)
gc.collect()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 128)               2560128   
_________________________________________________________________
dropout_36 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_37 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 23)                1495      
Total params: 2,569,879
Trainable params: 2,569,879
Non-trainable params: 0
_________________________________________________________________
None
Train on 36925 samples, validate on 4103 samples
Epoch 1/15
36925/36925 [==============================] - 6s - loss: 3.8629 - 

KeyboardInterrupt: 

# Basic RNN (LSTM)

In [29]:
model = keras.models.Sequential()
model.add(keras.layers.recurrent.LSTM(100, input_shape=input_shape))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(23, activation='softmax'))

In [30]:
model.compile(optimizer="adagrad",
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_split=0.1, epochs=15, batch_size=256)
yhat = model.predict(X_test)
_ = helper.score_prediction(y_test, yhat, binary=False)
gc.collect()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 100)               120400    
_________________________________________________________________
dropout_12 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 23)                2323      
Total params: 122,723
Trainable params: 122,723
Non-trainable params: 0
_________________________________________________________________
None
Train on 36925 samples, validate on 4103 samples
Epoch 1/15
36925/36925 [==============================] - 19s - loss: 2.6026 - categorical_accuracy: 0.2370 - val_loss: 2.4208 - val_categorical_accuracy: 0.2661
Epoch 2/15
36925/36925 [==============================] - 18s - loss: 2.4080 - categorical_accuracy: 0.2788 - val_loss: 2.2901 - val_categorical_accuracy: 0.3010
Epoch 3/15
36925/36925

/home/valentin/bin/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


313

# Basic RNN (GRU)

In [47]:
model = keras.models.Sequential()
model.add(keras.layers.recurrent.GRU(100, input_shape=input_shape))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(23, activation='softmax'))

In [48]:
model.compile(optimizer="adagrad",
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_split=0.1, epochs=15, batch_size=256)
yhat = model.predict(X_test)
_ = helper.score_prediction(y_test, yhat, binary=False)
gc.collect()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_14 (GRU)                 (None, 100)               90300     
_________________________________________________________________
dropout_23 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 23)                2323      
Total params: 92,623
Trainable params: 92,623
Non-trainable params: 0
_________________________________________________________________
None
Train on 36925 samples, validate on 4103 samples
Epoch 1/15
36925/36925 [==============================] - 16s - loss: 2.3841 - categorical_accuracy: 0.2942 - val_loss: 1.9515 - val_categorical_accuracy: 0.3883
Epoch 2/15
36925/36925 [==============================] - 16s - loss: 1.8802 - categorical_accuracy: 0.4022 - val_loss: 1.7801 - val_categorical_accuracy: 0.4241
Epoch 3/15
36925/36925 [

102

# Deep RNN (GRU)

In [45]:
model = keras.models.Sequential()
model.add(keras.layers.recurrent.GRU(64, input_shape=input_shape, return_sequences=True))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.recurrent.GRU(32))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(23, activation='softmax'))

In [46]:
model.compile(optimizer="adagrad",
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_split=0.1, epochs=15, batch_size=256)
yhat = model.predict(X_test)
_ = helper.score_prediction(y_test, yhat, binary=False)
gc.collect()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_12 (GRU)                 (None, 100, 64)           50880     
_________________________________________________________________
dropout_21 (Dropout)         (None, 100, 64)           0         
_________________________________________________________________
gru_13 (GRU)                 (None, 32)                9312      
_________________________________________________________________
dropout_22 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 23)                759       
Total params: 60,951
Trainable params: 60,951
Non-trainable params: 0
_________________________________________________________________
None
Train on 36925 samples, validate on 4103 samples
Epoch 1/15
36925/36925 [==============================] - 28s - loss: 2.5850 - categ

/home/valentin/bin/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


1135

# Basic CNN

In [108]:
filters = 100

model = keras.models.Sequential()
model.add(keras.layers.Conv1D(filters,
             kernel_size=1,
             padding='valid',
             activation='relu',
             strides=1, input_shape=input_shape))
model.add(keras.layers.pooling.GlobalMaxPooling1D())
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(23, activation='softmax'))

In [109]:
model.compile(optimizer="adagrad",
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_split=0.1, epochs=15, batch_size=256)
yhat = model.predict(X_test)
_ = helper.score_prediction(y_test, yhat, binary=False)
gc.collect()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_19 (Conv1D)           (None, 100, 100)          20100     
_________________________________________________________________
global_max_pooling1d_8 (Glob (None, 100)               0         
_________________________________________________________________
dropout_51 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_44 (Dense)             (None, 23)                2323      
Total params: 22,423
Trainable params: 22,423
Non-trainable params: 0
_________________________________________________________________
None
Train on 36925 samples, validate on 4103 samples
Epoch 1/15
36925/36925 [==============================] - 6s - loss: 2.1590 - categorical_accuracy: 0.3468 - val_loss: 1.8842 - val_categorical_accuracy: 0.4002
Epoch 2/15
36925/36925 [==============================]

/home/valentin/bin/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


102

# Recurrent CNN

In [7]:
input_shape = (100, 200)
filters = 100
kernel_size = 3
model = keras.models.Sequential()
model.add(keras.layers.Conv1D(filters,
             kernel_size,
             padding='valid',
             activation='relu',
             strides=1, input_shape=input_shape))
model.add(keras.layers.MaxPool1D(pool_size=10))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.recurrent.GRU(64))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(23, activation='softmax'))

In [111]:
model.compile(optimizer="adagrad",
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_split=0.1, epochs=15, batch_size=256)
yhat = model.predict(X_test)
_ = helper.score_prediction(y_test, yhat, binary=False)
gc.collect()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_20 (Conv1D)           (None, 98, 100)           60100     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 9, 100)            0         
_________________________________________________________________
dropout_52 (Dropout)         (None, 9, 100)            0         
_________________________________________________________________
gru_19 (GRU)                 (None, 64)                31680     
_________________________________________________________________
dropout_53 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 23)                1495      
Total params: 93,275
Trainable params: 93,275
Non-trainable params: 0
_________________________________________________________________
None
T

/home/valentin/bin/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


100